In [1]:
import matplotlib.pyplot as plt
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, Dataset, IterableDataset
import numpy as np
import multiprocessing as mp
import random
from torch.utils.data.sampler import Sampler

In [2]:
is_cuda = torch.cuda.is_available()
device = 'cuda' if is_cuda else 'cpu'
if not is_cuda:
    print("Warning: CUDA is not available, use CPU instead") 

- Total sample: $1020000$
- Train samples: $10^6$
- Validate samples: $10^4$
- Test samples: $10^4$
- batch_size: $2^{12} = 8192$
- num_batch: $120$; effective train samples: $120 * 8192 = 983040$
- rank_size: $122880 = 15 * 8192$, every time load: $15$ batches 
- num_rank: $120/15 = 8$, total $8$ times loading 


In [3]:
M = 2000
T = 1
tau = T/M
X0 = 0
V0 = 0.235**2
xi = 0.235**2
rho = -0.9
nu = 1.9
H = 0.07
batch_size = 2**13
num_batch = 120
P_train = num_batch * batch_size# size of train set 
P_valid = 10000
P_test = 10000 # size of test set 

rank_size = int(15 * batch_size) # sample size for every loading 
num_rank = int (num_batch/15) # number of loading 

# epochs = 10
# for test set, all moves to cpu
x0_t = torch.zeros(P_test, 1, device = device)
x0_b = torch.zeros(batch_size, 1, device = device)

logmoneyness = np.arange(-0.5, 0.31, 0.01)
strikes = np.exp(logmoneyness)

In [7]:
P_train

983040

In [4]:
S_path  = "/lustre1/u/u3553440/const_S/S_total"
V_path = "/lustre1/u/u3553440/V/V_total"
Z_path = "/lustre1/u/u3553440/Z/Z_total"

In [5]:
paths = [S_path, V_path, Z_path]

In [6]:
# load data 

class MyIterable(IterableDataset):
    
    def __init__(self, file_path: str, start, end):
        super(MyIterable, self).__init__()
        self.file_path = file_path        
        self.start = start
        self.end = end
   
    def __iter__(self):
        
        # single worker
        sample = []
        with open(self.file_path, 'r') as f:             
            for i, line in enumerate(f):
                if i < self.start: continue # 跳出本次循环，直到 i = start, 参与循环
                if i >= self.end: # 输出 i = [start, end-1] 
                    break # 跳出所有循环
                
                sample.append([float(i) for i in line.split()]) #将读取的字符串转为列表
            
        return np.array(sample)
        
    
    def __len__(self):
        num = 0 
        with open(self.file_path, 'r') as f:
            for _ in enumerate(f):
                num += 1
        return num 
 


In [ ]:
def MyDataset(start, rank_size, device, paths):
    end = start + rank_size
    # load data 
    ys = MyIterable(paths[0], start, end).__iter__()
    mul = MyIterable(paths[1], start, end).__iter__()
    z = MyIterable(paths[2], start, end).__iter__()
    
    # convert numpy array to tensor 
    ys = torch.from_numpy(ys).to(torch.float32).to(device)
    mul = np.c_[np.ones(rank_size), mul[:, :-1]]
    mul = torch.from_numpy(mul).to(torch.float32).to(device)
    z = torch.from_numpy(z).to(torch.float32).to(device)
    
    train_rank = TensorDataset(ys, mul, z)
    
    
    return train_rank

# train_rank_loader = DataLoader(train_rank, batch_size = batch_size, shuffle = True)

In [ ]:
# check the available cuda memory
def checkmem():
    free =  torch.cuda.mem_get_info()[0]//(1024**3)
    total = torch.cuda.mem_get_info()[1]//(1024**3)
    print(f"Free memory: {free:>5f}GB/ {total:>5f}GB")

In [ ]:
# load the validate set 
start_valid = 1000000
valid_set = MyDataset(start_valid, P_valid, device, paths)

In [ ]:
# load test set
start_test = 1010000
test_set = MyDataset(start_test, P_test, device, paths)

In [ ]:
checkmem()

In [ ]:
"""
# real samples
rbergomi_ys = np.load("SOE_2000_6_price.npy") # size (P, M) 

mul = np.load("SOE_2000_6_volatility.npy") # size (P, M)

# volatility at t_0 to t_{M-1}
mul = np.c_[np.ones(1000000), mul[:,:-1]]

# precomputed Brownian motion paths that drive the stock price
Z = np.load("SOE_2000_6_Bm.npy") # size (P, M)


# Convert numpy array to tensor
# only remain the first P paths 
rbergomi_ys = torch.from_numpy(rbergomi_ys).to(torch.float32).to(device)[:P, :]
mul = torch.from_numpy(mul).to(torch.float32).to(device)[:P, :]
Z = torch.from_numpy(Z).to(torch.float32).to(device)[:P, :]

# dataset 
dataset = TensorDataset(rbergomi_ys, mul, Z)
train_set, test_set = torch.utils.data.random_split(dataset, [P_train, P_test])

# train dataloader 
train_dataloader = DataLoader(train_set, batch_size = batch_size, shuffle = True)

"""

## Stock price 
$X(t) = logS(t)$

$dX(t) = -\frac{1}{2}V(t)dt + \sqrt{V(t)}dZ(t)$

where $X(0) = logS(0) = 0$

In [ ]:
class my_MLP(torch.nn.Module):
    def __init__(self, in_size, mlp_size, num_layers):
        # in_size: input size 
        # mlp_size: size of hidden layers 
        # num_layers: num of hidden layers 
        super().__init__()
    
        model = [torch.nn.Linear(in_size, mlp_size), torch.nn.LeakyReLU(0.1)]
        for _ in range(num_layers - 1):
            model.append(torch.nn.Linear(mlp_size, mlp_size))
            model.append(torch.nn.LeakyReLU(0.1))
            
        #output size: 1
        model.append(torch.nn.Linear(mlp_size, 1))
        # model.append(torch.nn.Tanh())
    
        self._model = torch.nn.Sequential(*model)
        
    def forward(self, x):
        return self._model(x) 
    

# numerical SDE solver 
# specifically applies to rough bergomi model 
# can only deal with the 1-dim BM case and use forward Euler method 
class sdeint:
    def __init__(self, neural_sde, x0, ts, mul, Z):
        
        # forward variance curve as neural sde 
        self.neural_sde = neural_sde
        # initial log stock price 
        self.x0 = x0 #(batch_size, )
        self.batch_size = x0.shape[0]        
        
        # discretized time grid
        self.ts = ts #(M, )
        self.num_grid = ts.shape[0] # =M
        self.tau = ts[1] - ts[0]
        
        # precomputed paths of volatility and Brownian motion  
        self.mul = mul
        self.Z = Z        
    
        
    def __call__(self):
        # forward Euler 
        neural_xs = torch.zeros(self.batch_size, self.num_grid + 1, device = device)       
        
        for i in range(1, self.num_grid+1):
            t = ts[i-1]
            V = self.neural_sde(t.reshape(-1,1)).squeeze(-1) * self.mul[:, i-1] #(batch_size, )
            neural_xs[:, i] = neural_xs[:, i-1] - V * self.tau/2 + torch.sqrt(V) * self.Z[:, i-1]
            
        return neural_xs[:, 1:] #(batch_size, M)


def price(ys, strikes):
    sample_size = ys.size(0) 
    strike_size = strikes.shape[0] # strikes is a 1-d array, has shape (strike_size, )    
    
    with torch.no_grad():
        y_T = ys[:, -1].cpu().numpy() #(P, )
        
    Y = np.tile(y_T, (strike_size, 1)) #(strike_size, P)
    K = np.tile(np.reshape(strikes, (-1, 1)), (1, sample_size)) #(strike_size, P)
    Y_K = Y - K
    Y_K[Y_K < 0] = 0
    price = np.mean(Y_K, -1) # 1-d array, has shape (strike_size, )
    return price


#plot the marginal distribution at T and the option price 
def my_plot(neural_ys, real_ys, neural_price, real_price, strikes):
    
    with torch.no_grad():
        neural_ys_1 = neural_ys[:, -1].cpu().numpy()            
        real_ys_1 = real_ys[:, -1].cpu().numpy()   
 
    plt.figure(figsize = (12, 5))
    plt.subplot(1,2,1)     
    _, bins, _ = plt.hist(neural_ys_1, bins = 100, alpha = 0.7, color = "crimson", density = True)
    bin_width = bins[1] - bins[0]
    num_bins = int((real_ys_1.max() - real_ys_1.min()) // bin_width)
    plt.hist(real_ys_1, bins = 100 , alpha = 0.7, color = "dodgerblue", density = True)
    plt.legend(["Neural SDE", "Real"], fontsize = 12)
    plt.xlabel("Value", fontweight = "heavy")
    plt.ylabel("Density", fontweight = "heavy")
    plt.title("Empirical distribution at t = T", fontsize = 14, fontweight = "heavy")
    plt.grid(True)
    
    plt.subplot(1,2,2)    
    plt.plot(strikes, neural_price, color = "crimson", lw = 2)
    plt.plot(strikes, real_price, color = "dodgerblue", lw = 2)
    plt.legend(["Neural SDE price", "Real price"], fontsize = 12)
    plt.xlabel("Strikes", fontweight = "heavy")
    plt.ylabel("Price", fontweight = "heavy")
    plt.title("Option price", fontsize = 14, fontweight = "heavy") 
    plt.grid(True)
    plt.show() 
    

# return the Wasserstein_p distance between two empircial ditributions 
def Wasserstein_p(real_ys, neural_ys, p):
    # real_ys, neural_ys have size (batch_size, M)
    
    real_ys_sorted, _ = torch.sort(real_ys, 0) #改变行，不改变列
    neural_ys_sorted, _ = torch.sort(neural_ys, 0)
    loss = torch.mean(torch.abs(real_ys_sorted - neural_ys_sorted)**p, 0)**(1/p) #(M, )
    return loss



In [ ]:
# Xavier initialization
def init_weights(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.001)       

forward_var.apply(init_weights)


In [ ]:
# neural_ys 
forward_var = my_MLP(1, 100, 10)
# forward_var = torch.load("/lustre1/u/u3553440/NN/const_after_2.pth")
forward_var = forward_var.to(device)

ts = torch.linspace(tau, T, M, device = device)
neural_ys = sdeint(forward_var, x0_t, ts, test_set[:P_test][1], test_set[:P_test][2])()
neural_ys = torch.exp(neural_ys)


In [ ]:
neural_ys[:, -1]

In [ ]:
checkmem()

In [ ]:
# torch.save(forward_var, "/lustre1/u/u3553440/NN/const_1.pth")

In [ ]:
#real_price
real_price = price(test_set[:P_test][0], strikes)

# neural_price 
neural_price = price(neural_ys, strikes)

# plot before training 
my_plot(neural_ys, test_set[:P_test][0], neural_price, real_price, strikes)


In [ ]:
Wasserstein_p(test_set[:int(P_test/2)][0], test_set[int(P_test/2):][0], p = 1)[-1]

In [ ]:
# forward_var = torch.load("ds_1000.pth")

In [ ]:
#optimization
my_optimizer = torch.optim.Adam(forward_var.parameters(), lr= 1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(my_optimizer, 'min', verbose = True, patience = 1)


In [ ]:
checkmem()


In [ ]:
loss_history = []
max_error_history = []
# Wasserstein-1 distance as loss function
epoch = 5
random_rank = list(range(num_rank))
random.shuffle(random_rank)
for i in range(epoch):    
    print(f"Epoch {i+1}\n-------------------------")
    for rank in random_rank:
        start = rank * rank_size
        train_rank = MyDataset(start, rank_size, device, paths)
        train_rank_loader = DataLoader(train_rank, batch_size = batch_size, shuffle = True)
    
        for batch, train_samples in enumerate(train_rank_loader):         
        
            real_samples, mul_samples, Z_samples = train_samples # (batch_size, M)
        
            neural_samples = sdeint(forward_var, x0_b, ts, mul_samples, Z_samples)()
            neural_samples = torch.exp(neural_samples) # (batch_size, M)
        
            # compute Wasserstein-1 distance at all time grids 
            loss = Wasserstein_p(real_samples, neural_samples, p = 1)[-1]
        
            # Wasserstein-1 distance at t = T
            loss_history.append(loss.item())
        
            # average Wasserstein distance at all time steps
            # ave_loss = torch.mean(loss)            
            price_error = np.abs(price(neural_samples, strikes) - price(real_samples, strikes))
            max_error_history.append(price_error.max())        

            my_optimizer.zero_grad()
            # use the Wasserstein distance at T as loss
            loss.backward()
            my_optimizer.step()        
         
            current = batch * batch_size 
            print(f"loss: {loss:>7f}  [{current:>5d}/{P_train:>5d}]")
        
    random.shuffle(random_rank)
    
    # check the model's performance on validate set after every epoch 
    
    neural_ys = sdeint(forward_var, x0_t, ts, test_set[:P_test][1], test_set[:P_test][2])()
    neural_ys = torch.exp(neural_ys)
        
    valid_loss = Wasserstein_p(test_set[:P_test][0], neural_ys, p = 1)[-1]
    scheduler.step(valid_loss)
    print(f"Valid Loss: {valid_loss:>7f}")
        
  

In [ ]:
# torch.save(forward_var,'const_1e.pth')

In [ ]:
# plot after training 
neural_samples = sdeint(forward_var, x0_t, ts, test_set[:P_test][1], test_set[:P_test][2])()
neural_samples = torch.exp(neural_samples)

neural_price_aftert = price(neural_samples, strikes)

my_plot(neural_samples, test_set[:P_test][0], neural_price_aftert, real_price, strikes)

In [ ]:
# total_num_batches = int(P_train/batch_size * 10)
# my_x = np.linspace(1, total_num_batches, total_num_batches)
#only the first 250 batches will be plotted 
my_x = np.linspace(1, num_batch * 5, num_batch * 5)
my_y_1 = np.array(loss_history)
my_y_2 = np.array(max_error_history)
plt.figure(figsize = (20, 5))
plt.plot(my_x, my_y_1, color = "dodgerblue", lw = 1)
plt.plot(my_x, my_y_2, color = "crimson", lw = 1)
# plt.legend(["Wasserstein-1 distance at T", "Max option price error"], fontsize = 12)
plt.xlabel("Batch number", fontweight = "heavy")
plt.yscale('log')
plt.title("Learning Curve", fontsize = 14, fontweight = "heavy")    
plt.show()  

In [ ]:
file = open('/lustre1/u/u3553440/NN/deep_loss_history_6','w')
file.write(str(loss_history))
file.close()


In [ ]:
file = open('/lustre1/u/u3553440/NN/deep_max_error_6','w')
file.write(str(max_error_history))
file.close()


In [ ]:
torch.save(forward_var, "/lustre1/u/u3553440/NN/deep_const_after_6.pth")

In [ ]:
my_batch_1 = []
my_batch_2 = []
for batch, train_samples in enumerate(train_dataloader):
    ys_samples, _, _ = train_samples 
    if batch % 2 == 0:
        my_batch_1.append(ys_samples)
    else:
        my_batch_2.append(ys_samples)
        

In [ ]:
loss_ = []
for i in range(8):
    loss_.append(Wasserstein_p(my_batch_1[i], my_batch_2[i], p = 1)[-1].item())

In [ ]:
import statistics
statistics.mean(loss_)

In [ ]:
my_x = np.linspace(1, 100 * 8, 100 * 8)
plot, axes = plt.subplots(figsize = (20, 6))
axes.plot(my_x, sgd_loss_5800[:800], ".:", color = "crimson", lw = 1)
axes.plot(my_x, sdg_error_5800[:800], ".:", color = "dodgerblue", lw = 1)

axes.set_xlabel("Batch number", fontsize=16)
# axes.set_ylabel(r'$err$', fontsize=16)
# axes.set_xscale('symlog')
axes.set_yscale('symlog', linthresh=0.001)
axes.legend(["Wasserstein-1 distance at T", "Max option price error"], fontsize= "large")
plt.title("Learning curve", fontsize = 14, fontweight = "heavy")    
plt.show()  

In [ ]:
sgd_loss_5800 = np.concatenate([np.load("loss_2000_sdg.npy"), np.load("loss_2800_sdg.npy"), np.array(loss_history)])
sdg_error_5800 = np.concatenate([np.load("error_2000_sdg.npy"), np.load("error_2800_sdg.npy"), np.array(max_error_history)])

In [ ]:
np.save("sgd_1oss_5800.npy", sgd_loss_5800)
np.save("sgd_error_5800.npy", sdg_error_5800)

### Learning curve 

In [ ]:
my_loss = np.load("sgd_1oss_5800.npy")
my_error = np.load("sgd_error_5800.npy")

In [ ]:
def save_file(file_name, file):
    file_path = "/lustre1/u/u3553440/figs//Learning curve/{name}".format(name = file_name)
    np.savetxt(file_path, file)

In [ ]:
save_file("loss_hist", my_loss)
save_file("error_hist", my_error)

In [ ]:
my_loss.shape